# 🤖 Transformer 모델 구현 실습

## 학부생을 위한 단계별 튜토리얼

이 노트북은 **"Attention is All You Need"** 논문의 Transformer를 
간단한 번역 태스크를 통해 이해하기 쉽게 구현합니다.

---

### 📚 학습 목표
1. Positional Encoding의 원리 이해
2. Multi-Head Attention 메커니즘 구현
3. Encoder-Decoder 구조 이해
4. 전체 Transformer 모델 학습

---

## 🔧 환경 설정

필요한 라이브러리를 설치하고 import 합니다.

In [ ]:
# 필요한 경우 PyTorch 설치 (Colab에는 이미 설치되어 있음)
# !pip install torch

# torch: 딥러닝 프레임워크 (텐서 연산, 자동 미분, GPU 가속)
import torch

# torch.nn: 신경망 구성 요소 (Layer, Loss function 등)
import torch.nn as nn

# torch.optim: 최적화 알고리즘 (Adam, SGD 등)
import torch.optim as optim

# math: 수학 함수 (sqrt, log 등)
import math

# numpy: 수치 연산 라이브러리
import numpy as np

# PyTorch 버전과 CUDA(GPU) 사용 가능 여부 확인
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

---

## 1️⃣ 단계 1: Positional Encoding (위치 인코딩)

Transformer는 RNN과 달리 **순서 정보가 없습니다**.  
따라서 단어의 위치 정보를 추가해주어야 합니다.

### 공식
$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

- `pos`: 위치 (0, 1, 2, ...)
- `i`: 차원 인덱스
- `d_model`: 임베딩 차원

In [ ]:
class PositionalEncoding(nn.Module):
    """
    Positional Encoding 클래스
    
    역할: 단어의 위치 정보를 임베딩에 추가
    이유: Transformer는 RNN과 달리 순서 정보가 없어서 위치를 명시적으로 알려줘야 함
    
    sin, cos 함수를 사용하여 각 위치마다 고유한 벡터를 만듭니다.
    """
    def __init__(self, d_model, max_len=5000):
        """
        Args:
            d_model (int): 임베딩 차원 (예: 512)
            max_len (int): 최대 시퀀스 길이 (예: 5000)
        
        이유: 미리 max_len 길이까지의 위치 인코딩을 계산해두면
              forward 시 매번 계산할 필요 없어 효율적임
        """
        super().__init__()
        
        # 위치 인코딩 행렬 생성: (max_len, d_model) 크기의 0으로 초기화된 텐서
        # 이유: 각 위치(행)마다 d_model 차원의 위치 벡터를 저장하기 위함
        pe = torch.zeros(max_len, d_model)
        
        # position: [0, 1, 2, ..., max_len-1]을 (max_len, 1) 형태로 생성
        # unsqueeze(1): (max_len,) -> (max_len, 1)로 차원 추가
        # 이유: broadcasting을 위해 열 벡터 형태로 만듦
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        # 주파수(frequency) 계산: 논문의 10000^(2i/d_model) 부분
        # torch.arange(0, d_model, 2): 0, 2, 4, ... (짝수 인덱스)
        # 이유: sin과 cos를 짝수/홀수 위치에 번갈아 적용하기 위함
        # exp(-log(10000) * 2i/d_model) = 10000^(-2i/d_model) = 1/10000^(2i/d_model)
        # 이유: 낮은 차원에서는 빠르게 변하고, 높은 차원에서는 천천히 변하는 주파수 생성
        #       -> 다양한 스케일의 위치 정보를 포착할 수 있음
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                            -(math.log(10000.0) / d_model))
        
        # sin 함수 적용: 짝수 인덱스 (0, 2, 4, ...)
        # pe[:, 0::2]: 모든 행의 짝수 열 선택
        # position * div_term: (max_len, 1) * (d_model/2,) -> (max_len, d_model/2)
        # 이유: 각 위치마다 다른 주파수의 sin 함수값을 계산
        pe[:, 0::2] = torch.sin(position * div_term)
        
        # cos 함수 적용: 홀수 인덱스 (1, 3, 5, ...)
        # 이유: sin과 cos를 번갈아 사용하면 위치를 더 명확하게 구분 가능
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 배치 차원 추가: (max_len, d_model) -> (1, max_len, d_model)
        # 이유: 입력 텐서가 (batch_size, seq_len, d_model) 형태이므로 
        #       broadcasting을 위해 배치 차원을 추가
        pe = pe.unsqueeze(0)
        
        # register_buffer: 모델의 파라미터는 아니지만 저장이 필요한 텐서 등록
        # 이유: 1) 학습되지 않는 고정값 2) model.save() 시 함께 저장됨 
        #       3) GPU/CPU 이동 시 자동으로 함께 이동
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        """
        Args:
            x (Tensor): 입력 임베딩, shape: (batch_size, seq_len, d_model)
        
        Returns:
            Tensor: 위치 정보가 추가된 임베딩, shape: (batch_size, seq_len, d_model)
        """
        # 입력 x에 위치 정보 더하기
        # self.pe[:, :x.size(1)]: 입력 시퀀스 길이만큼만 잘라서 사용
        # 이유: 입력 문장이 max_len보다 짧을 수 있으므로 필요한 길이만큼만 더함
        return x + self.pe[:, :x.size(1)]


# 테스트 코드
print("="*60)
print("Positional Encoding 테스트")
print("="*60)

# 테스트용 Positional Encoding 레이어 생성
# d_model=8: 작은 차원으로 테스트 (실제로는 512 정도 사용)
pe_layer = PositionalEncoding(d_model=8, max_len=10)

# 랜덤 샘플 입력 생성: (batch=2, seq_len=5, d_model=8)
# randn: 표준정규분포에서 샘플링한 랜덤 텐서
sample_input = torch.randn(2, 5, 8)

# forward 실행: 위치 정보가 추가된 출력
output = pe_layer(sample_input)

# 결과 확인
print(f"입력 shape: {sample_input.shape}")  # (2, 5, 8)
print(f"출력 shape: {output.shape}")        # (2, 5, 8) - shape는 동일
print(f"✓ Positional Encoding 완료!\n")

---

## 2️⃣ 단계 2: Multi-Head Attention (멀티헤드 어텐션)

여러 개의 attention head를 사용하여 **다양한 관점**에서 단어 간 관계를 학습합니다.

### 핵심 공식
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### 예시
문장: "나는 학교에 간다"
- 어떤 head는 **주어-동사** 관계에 집중
- 다른 head는 **목적어-동사** 관계에 집중

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention 클래스
    
    역할: 여러 개의 어텐션 헤드를 사용하여 입력의 다양한 측면을 병렬로 학습
    이유: 단일 어텐션보다 여러 관점에서 관계를 파악할 수 있어 성능이 좋음
    """
    def __init__(self, d_model, num_heads):
        """
        Args:
            d_model (int): 모델의 차원 (예: 512)
            num_heads (int): 어텐션 헤드 개수 (예: 8)
        
        이유: d_model을 num_heads로 나누어 각 헤드가 독립적으로 학습
        """
        super().__init__()
        
        # d_model이 num_heads로 나누어떨어지는지 확인
        # 이유: 각 헤드가 동일한 차원을 가져야 하므로
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        
        # 각 헤드의 차원 계산 (예: 512 / 8 = 64)
        # 이유: 전체 차원을 헤드 개수로 나누어 각 헤드가 담당할 차원 결정
        self.d_k = d_model // num_heads
        
        # Query, Key, Value를 만드는 선형 변환 레이어
        # Linear(d_model, d_model): 입력과 출력 차원이 같음
        # 이유: 입력을 Q, K, V로 변환하기 위한 학습 가능한 가중치 행렬
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        
        # 모든 헤드의 출력을 결합한 후 최종 변환하는 레이어
        # 이유: 각 헤드의 정보를 통합하여 최종 출력 생성
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        Scaled Dot-Product Attention 계산
        
        역할: Q(Query)와 K(Key)의 유사도를 계산하여 V(Value)에 가중치 적용
        
        직관적 설명:
        1. Q와 K를 곱해서 유사도 계산 (내적이 클수록 유사)
        2. softmax로 확률 분포 변환 (합이 1이 되도록)
        3. V에 가중치를 곱해서 최종 출력 (중요한 정보에 집중)
        
        Args:
            Q (Tensor): Query, shape: (batch, num_heads, seq_len, d_k)
            K (Tensor): Key, shape: (batch, num_heads, seq_len, d_k)
            V (Tensor): Value, shape: (batch, num_heads, seq_len, d_k)
            mask (Tensor): 어텐션 마스크 (선택적)
        
        Returns:
            output (Tensor): 어텐션 적용 결과
            attention_weights (Tensor): 어텐션 가중치 (시각화 가능)
        """
        # Q와 K의 내적으로 유사도 점수 계산
        # K.transpose(-2, -1): K의 마지막 두 차원을 전치 (seq_len, d_k) -> (d_k, seq_len)
        # matmul(Q, K^T): (batch, num_heads, seq_len_q, d_k) x (batch, num_heads, d_k, seq_len_k)
        #                 -> (batch, num_heads, seq_len_q, seq_len_k)
        # 이유: 각 Query가 모든 Key와 얼마나 유사한지 계산
        
        # math.sqrt(self.d_k)로 나누기 (Scaling)
        # 이유: 1) 내적 값이 너무 커지면 softmax의 gradient가 소실될 수 있음
        #       2) d_k가 클수록 내적 값의 분산이 커지므로 √d_k로 정규화
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # 마스크 적용 (선택적)
        # mask == 0인 위치를 -1e9(음의 무한대에 가까운 값)로 설정
        # 이유: 1) Decoder에서 미래 단어를 보지 못하게 함 (causal masking)
        #       2) padding 토큰을 무시 (padding mask)
        #       3) softmax 후 이 위치의 확률이 거의 0이 됨
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # softmax로 attention 가중치 계산
        # dim=-1: 마지막 차원(seq_len_k)에 대해 softmax 적용
        # 이유: 각 Query에 대해 모든 Key의 가중치 합이 1이 되도록
        #       -> 확률 분포로 해석 가능
        attention_weights = torch.softmax(scores, dim=-1)
        
        # 가중치를 V에 곱하기 (가중 평균)
        # (batch, num_heads, seq_len_q, seq_len_k) x (batch, num_heads, seq_len_k, d_k)
        # -> (batch, num_heads, seq_len_q, d_k)
        # 이유: 중요한 Value에 더 큰 가중치를 주어 정보 추출
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def forward(self, query, key, value, mask=None):
        """
        Multi-Head Attention의 forward pass
        
        Args:
            query, key, value (Tensor): shape: (batch_size, seq_len, d_model)
            mask (Tensor): 어텐션 마스크 (선택적)
        
        Returns:
            output (Tensor): 최종 출력, shape: (batch_size, seq_len, d_model)
            attention_weights (Tensor): 어텐션 가중치
        """
        batch_size = query.size(0)
        
        # 1단계: Linear 변환 후 multi-head로 split
        # Q = W_q * query: (batch, seq_len, d_model) -> (batch, seq_len, d_model)
        # view로 reshape: (batch, seq_len, num_heads, d_k)
        # 이유: d_model 차원을 (num_heads, d_k)로 분할하여 각 헤드가 독립적으로 처리
        # transpose(1, 2): (batch, seq_len, num_heads, d_k) -> (batch, num_heads, seq_len, d_k)
        # 이유: 헤드 차원을 앞으로 가져와 병렬 처리 용이
        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # 2단계: Scaled Dot-Product Attention 계산
        # 각 헤드가 독립적으로 어텐션 계산
        x, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # 3단계: Multi-head 결과 합치기 (Concatenation)
        # transpose(1, 2): (batch, num_heads, seq_len, d_k) -> (batch, seq_len, num_heads, d_k)
        # 이유: 원래 순서로 되돌림
        # contiguous(): 메모리 상에서 연속된 배치로 만듦
        # 이유: view 연산을 위해 필요 (transpose 후 메모리가 비연속적일 수 있음)
        # view(batch_size, -1, self.d_model): (batch, seq_len, num_heads * d_k)
        # 이유: 모든 헤드의 출력을 하나로 합침 (concat)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # 4단계: 최종 linear 변환
        # 이유: 결합된 헤드 정보를 다시 d_model 차원으로 변환하여 
        #       다음 레이어로 전달할 표현 생성
        output = self.W_o(x)
        
        return output, attention_weights


# 테스트 코드
print("="*60)
print("Multi-Head Attention 테스트")
print("="*60)

# Multi-Head Attention 레이어 생성
mha = MultiHeadAttention(d_model=512, num_heads=8)

# 랜덤 샘플 입력 생성: (batch=2, seq_len=10, d_model=512)
sample_input = torch.randn(2, 10, 512)

# Self-Attention 수행: query, key, value 모두 같은 입력 사용
# 이유: 문장 내 단어들끼리의 관계를 학습 (자기 자신을 참조)
output, attn_weights = mha(sample_input, sample_input, sample_input)

# 결과 확인
print(f"입력 shape: {sample_input.shape}")           # (2, 10, 512)
print(f"출력 shape: {output.shape}")                 # (2, 10, 512) - shape 동일
print(f"Attention weights shape: {attn_weights.shape}")  # (2, 8, 10, 10)
#     (batch, num_heads, seq_len, seq_len)
print(f"✓ Multi-Head Attention 완료!\n")

---

## 3️⃣ 단계 3: Feed Forward Network

간단한 2층 신경망입니다.  
각 단어 표현을 **독립적으로** 변환합니다.

$$\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2$$

In [ ]:
class FeedForward(nn.Module):
    """
    Feed Forward Network (Position-wise)
    
    역할: 각 위치(단어)의 표현을 독립적으로 비선형 변환
    이유: 1) 어텐션은 선형 변환이므로 비선형성을 추가하여 표현력 향상
          2) 각 단어의 표현을 더 풍부하게 만듦
    """
    def __init__(self, d_model, d_ff):
        """
        Args:
            d_model (int): 입력/출력 차원 (예: 512)
            d_ff (int): 중간 hidden layer 차원 (예: 2048)
        
        이유: d_ff는 보통 d_model의 4배 정도로 설정
              더 큰 차원에서 변환 후 다시 축소하여 표현력 증가
        """
        super().__init__()
        
        # 첫 번째 선형 레이어: d_model -> d_ff로 차원 확장
        # 이유: 더 큰 공간에서 복잡한 변환 수행
        self.linear1 = nn.Linear(d_model, d_ff)
        
        # 두 번째 선형 레이어: d_ff -> d_model로 차원 축소
        # 이유: 원래 차원으로 되돌려 다음 레이어로 전달
        self.linear2 = nn.Linear(d_ff, d_model)
        
        # ReLU 활성화 함수: max(0, x)
        # 이유: 비선형성 도입 (음수는 0으로, 양수는 그대로)
        #       간단하지만 효과적인 활성화 함수
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        Args:
            x (Tensor): 입력, shape: (batch_size, seq_len, d_model)
        
        Returns:
            Tensor: 출력, shape: (batch_size, seq_len, d_model)
        
        과정:
        1. linear1: (batch, seq_len, d_model) -> (batch, seq_len, d_ff)
        2. relu: 음수 값을 0으로 (비선형 변환)
        3. linear2: (batch, seq_len, d_ff) -> (batch, seq_len, d_model)
        
        중요: 각 위치(seq_len)마다 독립적으로 같은 변환 적용 (Position-wise)
        이유: 단어들 간의 상호작용은 어텐션에서 처리했으므로
              여기서는 각 단어의 표현만 개선
        """
        return self.linear2(self.relu(self.linear1(x)))


# 테스트 코드
print("="*60)
print("Feed Forward Network 테스트")
print("="*60)

# FFN 레이어 생성
ffn = FeedForward(d_model=512, d_ff=2048)

# 랜덤 샘플 입력
sample_input = torch.randn(2, 10, 512)
output = ffn(sample_input)

# 결과 확인
print(f"입력 shape: {sample_input.shape}")  # (2, 10, 512)
print(f"출력 shape: {output.shape}")        # (2, 10, 512) - shape 동일
print(f"✓ Feed Forward Network 완료!\n")

---

## 4️⃣ 단계 4: Encoder Layer (인코더 레이어)

Transformer Encoder의 기본 블록입니다.

### 구조
1. Multi-Head **Self-Attention**
2. Add & Norm (Residual Connection + Layer Normalization)
3. Feed Forward Network
4. Add & Norm

In [ ]:
class EncoderLayer(nn.Module):
    """
    Encoder Layer
    
    역할: 입력 문장을 이해하고 의미를 추출하는 인코더의 한 층
    구조: Self-Attention -> Add&Norm -> FFN -> Add&Norm
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Args:
            d_model (int): 모델 차원
            num_heads (int): 어텐션 헤드 개수
            d_ff (int): FFN의 hidden 차원
            dropout (float): Dropout 비율 (기본 0.1 = 10%)
        
        dropout 이유: 과적합(overfitting) 방지
                     학습 시 랜덤하게 일부 뉴런을 0으로 만들어 
                     모델이 특정 뉴런에 과도하게 의존하지 않도록 함
        """
        super().__init__()
        
        # Multi-Head Self-Attention
        # 이유: 문장 내 단어들 간의 관계 학습
        self.attention = MultiHeadAttention(d_model, num_heads)
        
        # Feed Forward Network
        # 이유: 각 단어의 표현을 비선형 변환
        self.feed_forward = FeedForward(d_model, d_ff)
        
        # Layer Normalization (정규화)
        # LayerNorm: 각 샘플의 feature들을 정규화 (평균 0, 분산 1)
        # 이유: 1) 학습 안정화 (gradient 폭발/소실 방지)
        #       2) 학습 속도 향상
        #       3) 초기화에 덜 민감
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # Dropout 레이어
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        Args:
            x (Tensor): 입력, shape: (batch_size, seq_len, d_model)
            mask (Tensor): 어텐션 마스크 (padding 무시용)
        
        Returns:
            Tensor: 인코더 레이어 출력, shape: (batch_size, seq_len, d_model)
        """
        
        # 1. Self-Attention + Residual Connection + Normalization
        # Self-Attention: query, key, value 모두 x 사용
        # 이유: 입력 문장의 각 단어가 다른 단어들과의 관계 파악
        attn_output, _ = self.attention(x, x, x, mask)
        
        # Residual Connection: x + dropout(attn_output)
        # 이유: 1) Gradient 흐름 개선 (깊은 네트워크 학습 가능)
        #       2) 원본 정보 보존 (정보 손실 방지)
        # Layer Normalization: 정규화하여 안정적인 학습
        # 순서: x + dropout(sublayer(x)) -> LayerNorm (Post-LN)
        x = self.norm1(x + self.dropout(attn_output))
        
        # 2. Feed Forward + Residual Connection + Normalization
        # FFN: 각 위치의 표현을 독립적으로 변환
        ff_output = self.feed_forward(x)
        
        # 다시 Residual Connection과 LayerNorm 적용
        # 이유: 위와 동일 (gradient 흐름, 정보 보존)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x


# 테스트 코드
print("="*60)
print("Encoder Layer 테스트")
print("="*60)

# Encoder Layer 생성
encoder_layer = EncoderLayer(d_model=512, num_heads=8, d_ff=2048)

# 랜덤 샘플 입력
sample_input = torch.randn(2, 10, 512)
output = encoder_layer(sample_input)

# 결과 확인
print(f"입력 shape: {sample_input.shape}")  # (2, 10, 512)
print(f"출력 shape: {output.shape}")        # (2, 10, 512) - shape 동일
print(f"✓ Encoder Layer 완료!\n")

---

## 5️⃣ 단계 5: Decoder Layer (디코더 레이어)

Transformer Decoder의 기본 블록입니다.

### 구조
1. Masked Multi-Head **Self-Attention** (자기 자신)
2. Add & Norm
3. Multi-Head **Cross-Attention** (Encoder 출력 참조)
4. Add & Norm
5. Feed Forward Network
6. Add & Norm

In [ ]:
class DecoderLayer(nn.Module):
    """
    Decoder Layer
    
    역할: 인코더의 출력을 참조하여 출력 문장을 순차적으로 생성
    구조: Masked Self-Attention -> Add&Norm -> Cross-Attention -> Add&Norm -> FFN -> Add&Norm
    
    Encoder와의 차이점:
    1. Masked Self-Attention: 미래 단어를 볼 수 없음 (autoregressive)
    2. Cross-Attention: 인코더 출력을 참조하여 입력 문장 정보 활용
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Args:
            d_model (int): 모델 차원
            num_heads (int): 어텐션 헤드 개수
            d_ff (int): FFN의 hidden 차원
            dropout (float): Dropout 비율
        """
        super().__init__()
        
        # Self-Attention: 출력 문장 내 단어들 간의 관계
        # 이유: 지금까지 생성한 단어들을 참조하여 다음 단어 예측
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        
        # Cross-Attention: 인코더 출력을 참조
        # 이유: 입력 문장(source)의 정보를 활용하여 출력(target) 생성
        #       예: "I love you" -> "나는 너를 사랑해" 번역 시
        #           "사랑해"를 생성할 때 "love"에 주목
        self.cross_attention = MultiHeadAttention(d_model, num_heads)
        
        # Feed Forward Network
        self.feed_forward = FeedForward(d_model, d_ff)
        
        # Layer Normalization (3개 필요)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        Args:
            x (Tensor): 디코더 입력 (target), shape: (batch, tgt_len, d_model)
            encoder_output (Tensor): 인코더 출력, shape: (batch, src_len, d_model)
            src_mask (Tensor): 소스 마스크 (padding 무시)
            tgt_mask (Tensor): 타겟 마스크 (padding + 미래 단어 무시)
        
        Returns:
            Tensor: 디코더 레이어 출력, shape: (batch, tgt_len, d_model)
        """
        
        # 1. Masked Self-Attention
        # tgt_mask 적용: 현재 위치보다 뒤에 있는 단어를 볼 수 없음
        # 이유: 학습 시 정답을 미리 보면 안 되므로 (Teacher Forcing)
        #       추론 시에도 순차적으로 생성하므로 미래 단어가 없음
        attn_output, _ = self.self_attention(x, x, x, tgt_mask)
        
        # Residual + LayerNorm
        x = self.norm1(x + self.dropout(attn_output))
        
        # 2. Cross-Attention (핵심!)
        # Query: 디코더의 현재 상태 (x)
        # Key, Value: 인코더 출력 (encoder_output)
        # 이유: "디코더가 현재 생성할 단어를 위해 입력 문장의 어디에 주목해야 하는가?"
        #       예: "나는"을 생성할 때 "I"에, "사랑해"를 생성할 때 "love"에 주목
        attn_output, _ = self.cross_attention(x, encoder_output, encoder_output, src_mask)
        
        # Residual + LayerNorm
        x = self.norm2(x + self.dropout(attn_output))
        
        # 3. Feed Forward Network
        # 각 위치의 표현을 독립적으로 변환
        ff_output = self.feed_forward(x)
        
        # Residual + LayerNorm
        x = self.norm3(x + self.dropout(ff_output))
        
        return x


# 테스트 코드
print("="*60)
print("Decoder Layer 테스트")
print("="*60)

# Decoder Layer 생성
decoder_layer = DecoderLayer(d_model=512, num_heads=8, d_ff=2048)

# 랜덤 샘플 입력
# tgt_input: 디코더 입력 (출력 문장)
# enc_output: 인코더 출력 (입력 문장의 표현)
tgt_input = torch.randn(2, 8, 512)   # 출력 문장 길이 8
enc_output = torch.randn(2, 10, 512)  # 입력 문장 길이 10

output = decoder_layer(tgt_input, enc_output)

# 결과 확인
print(f"Target 입력 shape: {tgt_input.shape}")     # (2, 8, 512)
print(f"Encoder 출력 shape: {enc_output.shape}")  # (2, 10, 512)
print(f"Decoder 출력 shape: {output.shape}")      # (2, 8, 512)
print(f"✓ Decoder Layer 완료!\n")

---

## 6️⃣ 단계 6: 완전한 Transformer 모델

이제 모든 컴포넌트를 결합하여 **완전한 Transformer**를 만듭니다!

### 전체 구조
```
Input → Embedding → Positional Encoding → Encoder (×N) → 
                                              ↓
Target → Embedding → Positional Encoding → Decoder (×N) → Linear → Output
```

In [ ]:
class Transformer(nn.Module):
    """
    완전한 Transformer 모델 (Sequence-to-Sequence)
    
    역할: 입력 시퀀스를 출력 시퀀스로 변환 (예: 기계 번역)
    구조: Encoder (입력 이해) + Decoder (출력 생성)
    """
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, 
                 num_heads=8, num_layers=6, d_ff=2048, dropout=0.1, max_len=5000):
        """
        Args:
            src_vocab_size (int): 소스(입력) 언어의 어휘 크기
            tgt_vocab_size (int): 타겟(출력) 언어의 어휘 크기
            d_model (int): 모델의 차원 (기본 512)
            num_heads (int): 어텐션 헤드 개수 (기본 8)
            num_layers (int): 인코더/디코더 레이어 개수 (기본 6)
            d_ff (int): FFN의 hidden 차원 (기본 2048)
            dropout (float): Dropout 비율 (기본 0.1)
            max_len (int): 최대 시퀀스 길이 (기본 5000)
        
        논문 "Attention is All You Need"의 기본 설정:
        d_model=512, num_heads=8, num_layers=6, d_ff=2048
        """
        super().__init__()
        
        # Embedding layers
        # nn.Embedding: 정수 인덱스를 고정 크기 벡터로 변환
        # 예: 단어 ID 5 -> [0.2, -0.1, 0.5, ..., 0.3] (d_model 차원 벡터)
        # 이유: 단어를 컴퓨터가 처리할 수 있는 수치 벡터로 표현
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        
        # Positional Encoding
        # 이유: 단어의 위치 정보 추가 ("첫 번째 단어", "두 번째 단어" 등)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        
        # Encoder layers (여러 층을 쌓음)
        # nn.ModuleList: 여러 레이어를 리스트로 관리 (파라미터 자동 등록)
        # 이유: 같은 구조의 레이어를 num_layers개 쌓아서 
        #       더 복잡한 패턴 학습 (깊은 네트워크)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # Decoder layers (여러 층을 쌓음)
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # 최종 출력 레이어
        # Linear: d_model -> tgt_vocab_size
        # 이유: 각 위치마다 타겟 어휘의 모든 단어에 대한 점수(logit) 계산
        #       예: [0.1, 2.3, -0.5, ...] (vocab_size 개의 점수)
        #       가장 높은 점수의 단어를 예측
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # d_model 저장 (embedding scaling에 사용)
        self.d_model = d_model
        
    def generate_mask(self, src, tgt):
        """
        마스크 생성 함수
        
        역할: 1) Padding 토큰 무시 2) 미래 단어 가리기
        
        Args:
            src (Tensor): 소스 시퀀스, shape: (batch, src_len)
            tgt (Tensor): 타겟 시퀀스, shape: (batch, tgt_len)
        
        Returns:
            src_mask: 소스 마스크 (batch, 1, 1, src_len)
            tgt_mask: 타겟 마스크 (batch, 1, tgt_len, tgt_len)
        """
        # Source Mask: padding 토큰(0) 위치를 False로
        # (src != 0): padding이 아닌 위치는 True
        # unsqueeze(1).unsqueeze(2): broadcasting을 위한 차원 추가
        # 이유: 어텐션에서 padding 위치를 무시하기 위함
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        
        # Target Mask: padding + 미래 단어 가리기
        tgt_len = tgt.size(1)
        
        # Padding mask
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        
        # Subsequent mask (하삼각 행렬)
        # torch.tril: 하삼각 행렬 생성 (대각선 포함)
        # [[1, 0, 0],
        #  [1, 1, 0],
        #  [1, 1, 1]]
        # 이유: i번째 단어는 0~i번째 단어만 볼 수 있음 (미래 못 봄)
        tgt_sub_mask = torch.tril(torch.ones((tgt_len, tgt_len))).bool().to(tgt.device)
        
        # 두 마스크를 AND 연산으로 결합
        # 이유: padding도 무시하고 미래도 가려야 함
        tgt_mask = tgt_mask & tgt_sub_mask
        
        return src_mask, tgt_mask
        
    def forward(self, src, tgt):
        """
        Forward pass
        
        Args:
            src (Tensor): 소스 시퀀스 (단어 ID), shape: (batch, src_len)
            tgt (Tensor): 타겟 시퀀스 (단어 ID), shape: (batch, tgt_len)
        
        Returns:
            Tensor: 출력 logits, shape: (batch, tgt_len, tgt_vocab_size)
        """
        
        # 마스크 생성
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        
        # ===== Encoder 부분 =====
        # 1. Source Embedding
        # src: (batch, src_len) -> (batch, src_len, d_model)
        src_embedded = self.src_embedding(src)
        
        # 2. Embedding Scaling
        # math.sqrt(self.d_model)을 곱함
        # 이유: 논문에서 제시한 방법, positional encoding과의 균형 유지
        #       임베딩 값의 스케일을 키워 위치 정보와 섞일 때 균형 맞춤
        src_embedded = src_embedded * math.sqrt(self.d_model)
        
        # 3. Positional Encoding 추가
        src_embedded = self.pos_encoding(src_embedded)
        
        # 4. Dropout 적용
        src_embedded = self.dropout(src_embedded)
        
        # 5. Encoder layers 통과
        # 각 레이어를 순차적으로 통과하며 점점 더 추상적인 표현 학습
        enc_output = src_embedded
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)
        
        # ===== Decoder 부분 =====
        # 1. Target Embedding + Scaling
        tgt_embedded = self.tgt_embedding(tgt) * math.sqrt(self.d_model)
        
        # 2. Positional Encoding 추가
        tgt_embedded = self.pos_encoding(tgt_embedded)
        
        # 3. Dropout 적용
        tgt_embedded = self.dropout(tgt_embedded)
        
        # 4. Decoder layers 통과
        # 각 레이어에서 encoder_output을 참조하여 출력 생성
        dec_output = tgt_embedded
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
        
        # ===== 최종 출력 =====
        # Linear layer로 vocab 크기의 logits 생성
        # (batch, tgt_len, d_model) -> (batch, tgt_len, tgt_vocab_size)
        # 이유: 각 위치에서 어떤 단어를 예측할지 점수 계산
        output = self.fc_out(dec_output)
        
        return output


# 모델 생성
print("="*60)
print("완전한 Transformer 모델 생성")
print("="*60)

# 하이퍼파라미터 설정
# 실제 애플리케이션에 맞게 조정 필요
src_vocab_size = 1000   # 소스 언어 어휘 크기
tgt_vocab_size = 1000   # 타겟 언어 어휘 크기
d_model = 256           # 작게 설정 (학습 속도를 위해, 실제로는 512)
num_heads = 8           # 어텐션 헤드 개수
num_layers = 3          # 인코더/디코더 레이어 수 (실제로는 6)
d_ff = 512              # FFN hidden 차원 (실제로는 2048)

# Transformer 모델 인스턴스 생성
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, 
                   num_heads, num_layers, d_ff)

# 파라미터 개수 계산
# 이유: 모델 크기 확인, 메모리 사용량 예측
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ 모델 생성 완료!")
print(f"- 전체 파라미터 수: {total_params:,}")
print(f"- 학습 가능 파라미터 수: {trainable_params:,}")
print(f"- 모델 크기: 약 {total_params * 4 / (1024**2):.2f} MB (float32 기준)\n")
# float32: 각 파라미터가 4바이트

---

## 7️⃣ 단계 7: 모델 학습 예제

이제 실제로 Transformer 모델을 **학습**해봅시다!

### 학습 과정
1. 데이터 준비 (더미 데이터)
2. Loss 함수 & Optimizer 설정
3. Training Loop
4. 결과 확인

In [ ]:
def train_transformer():
    """
    Transformer 모델 학습 함수
    
    역할: 모델을 학습시켜 번역(또는 다른 seq2seq 태스크) 수행 능력 획득
    """
    print("="*60)
    print("Transformer 모델 학습 시작")
    print("="*60)
    
    # 디바이스 설정 (GPU가 있으면 GPU, 없으면 CPU 사용)
    # 이유: GPU를 사용하면 학습 속도가 10~100배 빠름
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n사용 디바이스: {device}")
    
    # 모델을 디바이스로 이동
    # 이유: 모델과 데이터가 같은 디바이스에 있어야 연산 가능
    model.to(device)
    
    # 손실 함수 (Loss Function)
    # CrossEntropyLoss: 분류 문제에 사용되는 손실 함수
    # 이유: 각 위치에서 vocab_size개 단어 중 하나를 선택하는 분류 문제
    # ignore_index=0: padding 토큰(ID=0)은 손실 계산에서 제외
    # 이유: padding은 실제 단어가 아니므로 예측할 필요 없음
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    
    # 옵티마이저 (Optimizer)
    # Adam: Adaptive Moment Estimation, 가장 널리 사용되는 옵티마이저
    # 이유: 1) 학습률을 자동으로 조정 2) 빠른 수렴 3) 구현 간단
    # lr=0.0001: 학습률 (learning rate)
    # betas=(0.9, 0.98): Adam의 momentum 파라미터 (논문 설정)
    # eps=1e-9: 수치 안정성을 위한 작은 값
    optimizer = optim.Adam(model.parameters(), lr=0.0001, 
                          betas=(0.9, 0.98), eps=1e-9)
    
    # 학습 데이터 설정
    # 실제로는 토큰화된 실제 문장을 사용해야 함
    batch_size = 32      # 한 번에 처리할 문장 쌍의 개수
    src_seq_len = 10     # 소스 문장 길이
    tgt_seq_len = 10     # 타겟 문장 길이
    
    print(f"\n학습 설정:")
    print(f"- Batch size: {batch_size}")
    print(f"- Source sequence length: {src_seq_len}")
    print(f"- Target sequence length: {tgt_seq_len}")
    print(f"- Learning rate: 0.0001")
    
    # 학습 루프
    num_epochs = 10  # 전체 데이터를 몇 번 반복할지
    print(f"\n학습 시작 (Epochs: {num_epochs})...")
    print("-" * 60)
    
    # model.train(): 학습 모드로 설정
    # 이유: Dropout, BatchNorm 등이 학습 시에만 작동
    model.train()
    
    for epoch in range(num_epochs):
        # 더미 배치 생성 (실제로는 DataLoader 사용)
        # randint(1, 100, ...): 1~99 사이의 랜덤 정수 (0은 padding)
        # 이유: 실제 학습 데이터 대신 랜덤 데이터로 동작 확인
        src = torch.randint(1, 100, (batch_size, src_seq_len)).to(device)
        tgt = torch.randint(1, 100, (batch_size, tgt_seq_len)).to(device)
        
        # Gradient 초기화
        # 이유: 이전 배치의 gradient가 누적되지 않도록
        optimizer.zero_grad()
        
        # Forward pass (Teacher Forcing)
        # tgt[:, :-1]: 마지막 토큰 제외 (디코더 입력)
        # 이유: t 시점의 입력으로 t+1 시점의 출력 예측
        #       예: "<start> 나는 너를" -> "나는 너를 사랑해"
        output = model(src, tgt[:, :-1])
        
        # 손실 계산을 위한 reshape
        # output: (batch, tgt_len-1, vocab_size) -> (batch*(tgt_len-1), vocab_size)
        # 이유: CrossEntropyLoss는 2D 입력 필요
        output = output.reshape(-1, tgt_vocab_size)
        
        # target: tgt[:, 1:] (첫 토큰 제외)
        # 이유: t 시점의 입력에 대한 정답은 t+1 시점의 토큰
        target = tgt[:, 1:].reshape(-1)
        
        # 손실 계산
        # 이유: 예측과 정답의 차이를 계산하여 모델 개선 방향 파악
        loss = criterion(output, target)
        
        # Backward pass (역전파)
        # 이유: 손실에 대한 각 파라미터의 gradient 계산
        loss.backward()
        
        # Gradient clipping
        # clip_grad_norm_: gradient의 norm을 최대 1.0으로 제한
        # 이유: Gradient explosion 방지 (gradient가 너무 커지면 학습 불안정)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # 옵티마이저 스텝 (파라미터 업데이트)
        # 이유: 계산된 gradient를 사용하여 파라미터를 조금씩 수정
        #       θ_new = θ_old - lr * gradient
        optimizer.step()
        
        # 진행 상황 출력 (2 에포크마다)
        if (epoch + 1) % 2 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1:2d}/{num_epochs}] | Loss: {loss.item():.4f}")
    
    print("-" * 60)
    print("\n✓ 학습 완료!")
    print("\n💡 참고: 이것은 더미 데이터를 사용한 예제입니다.")
    print("   실제 번역 태스크를 위해서는:")
    print("   1. 실제 번역 데이터셋 (예: WMT, Multi30k)")
    print("   2. 적절한 토큰화 (Tokenization)")
    print("   3. 충분한 학습 에포크 (수십~수백 에포크)")
    print("   4. Learning rate scheduling (학습률 조정)")
    print("   5. Validation set으로 평가")


# 학습 실행
train_transformer()

---

## 📊 핵심 개념 정리

### 1. Self-Attention
- 문장 내 단어들 간의 관계를 학습
- Q, K, V를 이용한 유사도 계산
- **"어떤 단어가 어떤 단어와 관련있는가?"**

### 2. Multi-Head Attention
- 여러 관점에서 관계 학습
- 각 head가 다른 패턴을 학습
- **"다양한 방식으로 문장을 이해"**

### 3. Positional Encoding
- 단어의 순서 정보 추가
- sin, cos 함수 이용
- **"첫 번째 단어인지, 마지막 단어인지"**

### 4. Encoder-Decoder 구조
- **Encoder**: 입력 문장을 이해
- **Decoder**: 출력 문장을 생성
- **Cross-Attention**: Decoder가 Encoder 정보 참조

### 5. Residual Connection & Layer Normalization
- 깊은 네트워크의 학습을 안정화
- Gradient vanishing 문제 완화
- **"정보의 흐름을 원활하게"**

---

## 🚀 다음 단계

### 실습 과제
1. **Attention Weights 시각화**
   - 어떤 단어가 어떤 단어에 주목하는지 확인
   - Heatmap으로 표현

2. **실제 데이터로 학습**
   - Multi30k 데이터셋 (영어-독일어)
   - 간단한 영어-한국어 번역

3. **Beam Search 구현**
   - Greedy decoding 개선
   - 더 좋은 번역 결과

### 심화 학습
- BERT (Encoder only)
- GPT (Decoder only)
- Vision Transformer (이미지에 적용)

---

## 📚 참고 자료

1. **논문**: "Attention is All You Need" (Vaswani et al., 2017)
2. **블로그**: The Illustrated Transformer by Jay Alammar
3. **코드**: The Annotated Transformer
4. **강의**: CS224N (Stanford NLP)

---

### 🎉 축하합니다!

Transformer의 모든 핵심 컴포넌트를 구현했습니다!  
이제 여러분은 현대 NLP의 기반을 이해하게 되었습니다. 🚀

**"Attention is All You Need" - 그리고 당신의 노력도!** 💪

---

## 💻 추가 실습 문제

아래 셀에서 직접 코드를 작성해보세요!

In [ ]:
# 실습 1: Attention weights를 시각화해보세요
# 힌트: matplotlib의 imshow나 seaborn의 heatmap 사용

import matplotlib.pyplot as plt
import seaborn as sns

# TODO: 여기에 코드를 작성하세요


In [ ]:
# 실습 2: 간단한 추론(inference) 함수를 만들어보세요
# 입력: "I love you"
# 출력: "나는 너를 사랑해" (토큰 ID로)

def translate(model, src_sentence, max_len=20):
    """
    문장을 번역하는 함수
    
    Args:
        model: 학습된 Transformer 모델
        src_sentence: 입력 문장 (토큰 ID 리스트)
        max_len: 최대 생성 길이
    
    Returns:
        번역 결과 (토큰 ID 리스트)
    """
    # TODO: 여기에 코드를 작성하세요
    # 힌트: 1) model.eval() 모드로 전환
    #       2) torch.no_grad() 사용
    #       3) <start> 토큰부터 시작해서 한 단어씩 생성
    #       4) <end> 토큰이 나오거나 max_len에 도달하면 종료
    pass


In [ ]:
# 실습 3: Positional Encoding의 패턴을 시각화해보세요
# 힌트: plt.imshow를 사용하여 (max_len, d_model) 행렬 시각화

# TODO: 여기에 코드를 작성하세요
